In [38]:
from ipywidgets import interact, IntSlider
import os
import nibabel as nib
import eval_seg
eval_seg.reload()

import numpy as np

root = 'datasets/alg_out/'
root_ct = 'datasets/ct'
root_out = 'results'
import importlib

def _load(file):
    data,voxelsize=eval_seg.io.read_nib(file)
    return data,voxelsize
    

def _compute(dataset, data, ax,frame,name):
    
    
    gt_orig,voxelsize_orig = data['GroundTruth']
    

    
    
    ct_orig = gt_orig if data['CT'][0] is None else data['CT'][0]
    

    # outdir1 = f'{root_out}/{dataset}/{file}/'
    # os.makedirs(outdir1, exist_ok=True)

    data_orig = {typ: data[typ][0] for typ in data if typ!='CT'}
    # data_orig = {typ: data[typ][0] for typ in ['GroundTruth','CE','TopK10']}
    

    # eval_seg.ui.ortho_slicer(ct_orig, data_orig, gtmax, args={'show': 0, 'dst': f'{outdir1}/ortho_slice'})
    all_axes = {'x': 0, 'y': 1, 'z': 2, 'all': -1}

    
    classes=int(max(gt_orig.max(),max([data_orig[typ].max() for typ in data_orig]))+1)
    # print('number of classes:',classes)
    res={}
    
    
    axi = all_axes[ax]
    try:
        gt, voxelsize = eval_seg.geometry.slice(gt_orig, voxelsize_orig, axi, [frame])
        print(voxelsize_orig,'-->',voxelsize)
        ct,_ = eval_seg.geometry.slice(ct_orig, voxelsize_orig, axi, [frame])
        data = {p: eval_seg.geometry.slice(data_orig[p], voxelsize_orig, axi, [frame])[0] for p in data_orig}
        eval_seg.ui.multi_plot_2d(
                ct,
                data,
                spacing=voxelsize,
                args={
                    "imglabel": f"{dataset.split('_')[1]} {name} {ax}={frame}",
                    'z_titles':[frame],
                    "add_backimg": 1,
                    "clahe": 1,
                    "crop2roi": 1,
                    "zoom2segments": 1,
                    "add_notzoom_img": 1,
                    #                 'interactive':1,
                    "show_tp_fp_fn": 0,
                    "show": 1,
                    # "dst": f"{outdir2}/allmethods",
                    "spacing": voxelsize,
                },
            )


    except Exception as e:
        print(dataset, name, axes)
        print({d: data_orig[d].sum() for d in data_orig})
        raise e
    return res


@interact
def selectData(dataset=['Task001_LiverTumor','Task002_Pancreas','Task003_Multiorgan']):
    
    # def fileviewer(file=sorted([f.replace('_0000.','.') for f in os.listdir(f'{root_ct}/{dataset}/')])):       
    @interact
    def axselect(ax=['z','x','y']):       
        axi={'x':0,'y':1,'z':2}[ax]
        @interact
        def fileviewer(file=sorted([f for f in os.listdir(f'{root}/{dataset}/GroundTruth/')])):   
            files={k:f'{root}/{dataset}/{k}/{file}' for k in ['GroundTruth', 'CE', 'Dice', 'DiceCE', 'DiceFocal', 'DiceTopK10', 'TopK10']}
            files['CT']=f'{root_ct}/{dataset}/{file}'.replace('.nii.gz', '_0000.nii.gz')
            files_inv={files[k]:k for k in files}
            data={files_inv[k]:v for k,v in eval_seg.common.parallel_runner(_load,list(files.values()))}
            gt_orig,voxelsize_orig=data['GroundTruth']
            gtmax=(gt_orig.sum(axis=2).sum(axis=1).argmax(),gt_orig.sum(axis=2).sum(axis=0).argmax(),gt_orig.sum(axis=1).sum(axis=0).argmax())      
            @interact
            def onlyMaxFrame(frame=IntSlider(gtmax[axi],min=0,max=gt_orig.shape[axi]-1)):
                _compute(dataset,data,ax,frame,name=dataset+file)    

        

interactive(children=(Dropdown(description='dataset', options=('Task001_LiverTumor', 'Task002_Pancreas', 'Task…